In [1]:
##
# file:  ipm.R
#

##
# Interior-Point Method for Quadratic Programming.
#
# Quadratic program formulation, primal:
#   minimize  c^T x + \frac{1}{2} x^T Q x
#   subject to
#             Ax >= b
#             x >= 0
#
# Quadratic program formulation, dual:
#   maximize  b^T y - \frac{1}{2} x^T Q x
#   subject to
#             A^T y + z - Qx = c
#             y, z >= 0
#

##
# computes and returns a primal infeasibility measure
# (maximum relative violation of the primal constraints).
#
primalInfeasibility <- function(Ax, b) {
  primalInf <- 0
  d <- 0

  for (elem in seq_len(length(b))) {
    constraint <- Ax[elem]
    B <- b[elem]

    if (constraint < B) {
      d <- (B - constraint) / (abs(B) + 0.000001)
      if (primalInf < d) {
        primalInf <- d
      }
    }
  }

  primalInf
}

##
# computes and returns a dual infeasibility measure
# (maximum relative violation of the dual constraints).
#
dualInfeasibility <- function(ATy, z, Qx, c) {
  dualInf <- 0

  for (elem in seq_len(length(c))) {
    constraint <- ATy[elem] + z[elem] - Qx[elem]
    C <- c[elem]

    dualInf <- abs(constraint - C) / (abs(C) + 0.000001)
  }

  dualInf
}

##
# inner product of two vectors.
#
dot <- function(a, b) {
  d <- 0
  for (elem in seq_len(length(a))) {
    d <- d + a[elem] * b[elem]
  }
  d
}

##
# estimates and returns the penalty "mu"
# (see theory on the interior-point methods).
#
computeMu <- function(x, z, y, w) {
  nRows <- length(y)
  nColumns <- length(x)
  delta <- 0.1

  mu <- delta * (dot(z, x) + dot(y, w)) / (nRows + nColumns)
  mu
}

matrixVectorProduct <- function(A, x) {
  A %*% x
}

transposeVectorProduct <- function(A, y) {
  t(A) %*% y
}

quadVectorProduct <- function(Q, x) {
  if (is.matrix(Q)) {
    # fully dense matrix.
    Q %*% x
  } else {
    # otherwise it is a simple main diagonal stored as a vector.
    Q * x
  }
}

ipm <- function(Q, A, c, b) {
  # model dimensions
  nRows <- nrow(A)
  nColumns <- ncol(A)

  # user-controlled optimization parameters.
  maxIterations <- 100L
  maxPrimalInfeasibility <- 1e-4
  maxDualInfeasibility <- 1e-4
  maxObjRelDifference <- 1e-4

  x <- 10 * (1 + runif(nColumns))
  z <- 10 * (1 + runif(nColumns))
  deltaZ <- rep_len(0, length.out = nColumns)

  y <- 10 * (1 + runif(nRows))
  w <- 10 * (1 + runif(nRows))
  deltaW <- rep_len(0, length.out = nRows)

  cat(" Iter  Primal Objective    Dual Objective   Primal Inf     Dual Inf           Mu\n")

  for (iteration in seq_len(maxIterations)) {

    Ax <- matrixVectorProduct(A, x)
    ATy <- transposeVectorProduct(A, y)
    Qx <- quadVectorProduct(Q, x)
    xQx <- dot(x, Qx)
    mu <- computeMu(x, z, y, w)

    primalObjValue <- dot(c, x) + 0.5 * xQx
    dualObjValue <- dot(b, y) - 0.5 * xQx

    primalInf <- primalInfeasibility(Ax, b)
    dualInf <- dualInfeasibility(ATy, z, Qx, c)

    log <- sprintf("%5d  % 16.8e  % 16.8e  % 11.3e  % 11.3e  % 11.3e\n",
    iteration, primalObjValue, dualObjValue, primalInf, dualInf, mu)
    cat(log)

    objDiff <- abs(primalObjValue - dualObjValue) / (abs(primalObjValue) + 1)
    if (primalInf < maxPrimalInfeasibility &&
      dualInf < maxDualInfeasibility &&
      objDiff < maxObjRelDifference) {
      status <- "OK"
      break
    } else if (iteration == maxIterations) {
      status <- "MAX_ITERATIONS"
      break
    }

    if (is.matrix(Q)) {
      # fully dense matrix.
      kktUpper <- cbind(-diag(z / x) - Q, t(A))
    } else {
      # otherwise it is a simple main diagonal stored as a vector.
      kktUpper <- cbind(diag(- (z / x + Q), nrow = nColumns, ncol = nColumns), t(A))
    }

    kktLower <- cbind(A, diag(w / y, nrow = nRows, ncol = nRows))
    kkt <- rbind(kktUpper, kktLower)
    rhsUpper <- c - ATy - mu / x + Qx
    rhsLower <- b - Ax + mu / y
    rhs <- rbind(rhsUpper, rhsLower)
    ans <- solve(kkt, rhs)
    deltaX <- ans[1 : nColumns]
    deltaY <- ans[(nColumns + 1) : (nColumns + nRows)]

    # compute deltaZ
    for (elem in seq_len(nColumns)) {
      X <- x[elem]
      Z <- z[elem]
      deltaZ[elem] <- (mu - X * Z - Z * deltaX[elem]) / X
    }

    # compute deltaW
    for (elem in seq_len(nRows)) {
      Y <- y[elem]
      W <- w[elem]
      deltaW[elem] <- (mu - Y * W - W * deltaY[elem]) / Y
    }

    stepPrimal <- max(max(c(- deltaX / x, - deltaW / w)) / 0.95, 1)
    stepPrimal <- 1 / stepPrimal

    stepDual <- max(max(c(- deltaY / y, - deltaZ / z)) / 0.95, 1)
    stepDual <- 1 / stepDual

    for (elem in seq_len(nColumns)) {
      x[elem] <- x[elem] + stepPrimal * deltaX[elem]
      z[elem] <- z[elem] + stepDual * deltaZ[elem]
    }

    for (elem in seq_len(nRows)) {
      y[elem] <- y[elem] + stepDual * deltaY[elem]
      w[elem] <- w[elem] + stepPrimal * deltaW[elem]
    }
  }

  cat(sprintf("\nsolution status '%s'\n", status))
  cat("\nprimal solution vector  ")
  cat(x)

  cat("\ndual solution vector    ")
  cat(y)
  cat("\n")

  x
}

In [2]:
 ##################################################################
 ##################################################################
 c <- 0.5 * c(1, 1, 1, 1)
 Q <- c(1, 1, 1, 1)
 b <- c(1)
 A <- matrix(c(1, 2, 3, 4), nrow = 1L, byrow = FALSE)
 ipm(Q, A, c, b)

 ##################################################################
 ##################################################################
 c <- 0.5 * c(1, 1, 1, 1)
 Q <- c(1, 1, 1, 1)
 b <- c(1, 10)
 A <- matrix(c(1, 2, 3, 4, 4, 3, 2, 1), nrow = 2L, byrow = TRUE)
 ipm(Q, A, c, b)


 ##################################################################
 ##  NOW let us solve fully dense Q matrix.
 ##  remember - it needs to be POSITIVE SEMIDEFINITE.
 ##################################################################
 c <- 0.5 * c(1, 1, 1, 1)

 Q <- matrix(
   c(10, 1, 1, 1,
     1, 10, 1, 1,
     1, 1, 10, 1,
     1, 1, 1, 10),
   nrow = 4, ncol = 4, byrow = TRUE)

 b <- c(1, 10)
 A <- matrix(c(1, 2, 3, 4, 4, 3, 2, 1), nrow = 2L, byrow = TRUE)
 ipm(Q, A, c, b)

 Iter  Primal Objective    Dual Objective   Primal Inf     Dual Inf           Mu
    1    3.87398176e+02   -3.45466553e+02    0.000e+00    1.377e+02    2.365e+01
    2    5.09433308e+01   -4.26211733e+01    0.000e+00    4.690e+01    5.148e+00
    3    2.87979082e+01   -2.28398505e+01    0.000e+00    1.691e+00    1.151e+00
    4    1.05524623e+01   -6.97659126e+00    0.000e+00    2.220e-16    3.506e-01
    5    3.56897816e+00   -1.71213476e+00    0.000e+00    4.441e-16    1.056e-01
    6    1.22251755e+00   -3.28937805e-01    0.000e+00    3.331e-16    3.103e-02
    7    4.11027109e-01   -1.12545051e-02    0.000e+00    0.000e+00    8.446e-03
    8    2.39545460e-01    6.48211975e-02    0.000e+00    3.378e-02    3.206e-03
    9    1.84398367e-01    1.36725211e-01    0.000e+00    3.436e-02    9.072e-04
   10    1.62630107e-01    1.51550104e-01    0.000e+00    2.220e-16    2.216e-04
   11    1.56252428e-01    1.54124456e-01    0.000e+00    0.000e+00    4.256e-05
   12    1.55163047e-01    1

[1] 2.017943e-05 4.992794e-05 4.014241e-02 2.198720e-01

 Iter  Primal Objective    Dual Objective   Primal Inf     Dual Inf           Mu
    1    4.27679642e+02   -2.61562222e+02    0.000e+00    1.257e+02    2.085e+01
    2    2.69977506e+01    8.92308570e-01    0.000e+00    3.249e+01    3.448e+00
    3    2.21486291e+01   -1.22130067e+01    0.000e+00    6.102e-01    6.273e-01
    4    8.83493379e+00   -2.44701648e+00    0.000e+00    1.776e-15    1.880e-01
    5    3.98844466e+00    1.04208984e+00    0.000e+00    0.000e+00    4.911e-02
    6    3.42136321e+00    2.71408209e+00    0.000e+00    9.498e-02    1.041e-02
    7    3.28352574e+00    3.12894113e+00    0.000e+00    4.441e-16    2.576e-03
    8    3.25733454e+00    3.22550604e+00    0.000e+00    0.000e+00    5.305e-04
    9    3.25182332e+00    3.24617041e+00    0.000e+00    1.110e-16    9.422e-05
   10    3.25044172e+00    3.24926612e+00    0.000e+00    1.110e-16    1.959e-05
   11    3.25011146e+00    3.24982922e+00    0.000e+00    0.000e+00    4.704e-06

solution status 'OK'

prima

[1] 1.49814738 0.99862664 0.49911890 0.01333186

 Iter  Primal Objective    Dual Objective   Primal Inf     Dual Inf           Mu
    1    7.51088476e+03   -7.27664190e+03    0.000e+00    2.161e+02    2.349e+01
    2    1.05671551e+02   -6.56538194e+01    0.000e+00    4.263e-14    2.855e+00
    3    4.01523930e+01   -8.32288607e+00    0.000e+00    7.105e-15    8.079e-01
    4    2.43284946e+01    1.11321164e+01    0.000e+00    1.043e+00    1.970e-01
    5    2.22755556e+01    1.96385594e+01    0.000e+00    2.607e-01    4.400e-02
    6    2.18454308e+01    2.12800032e+01    0.000e+00    1.776e-15    9.424e-03
    7    2.17396169e+01    2.16209407e+01    0.000e+00    0.000e+00    1.978e-03
    8    2.17190699e+01    2.16937030e+01    0.000e+00    0.000e+00    4.228e-04
    9    2.17159993e+01    2.17114053e+01    0.000e+00    1.776e-15    7.657e-05
   10    2.17155941e+01    2.17150324e+01    0.000e+00    8.882e-16    9.362e-06

solution status 'OK'

primal solution vector  1.46548 0.9999736 0.5344716 0.06923251
dual solution vector   

[1] 1.46548041 0.99997365 0.53447158 0.06923251